In [ ]:
###############   DEPENDENCY LOAD BLOCK   ###############
# Designed for Google Colab with GPU support

import sys, subprocess, importlib, os

def pipi(args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + args)

# Prevent torchvision::nms bug (safe even if torchvision not used)
os.environ["TORCHVISION_DISABLE_NMS_EXPORT"] = "1"

# --- Core check
import numpy as np
print("Using NumPy:", np.__version__)
try:
    import torch
    print("Using Torch:", torch.__version__)
except Exception:
    pass

# --- Install neuralforecast and dependencies
pipi(["neuralforecast==1.7.4", "--no-deps"])
pipi([
    "pytorch-lightning==2.2.5",
    "torchmetrics==1.3.1",
    "einops>=0.7.0",
    "pandas>=2.2.0",
    "scikit-learn>=1.3.0",
    "lightgbm>=4.3.0"
])
pipi(["utilsforecast==0.2.12"])
pipi(["coreforecast==0.0.12"])

# --- Import checks
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import PMM, GMM, MAE as NF_MAE
from lightgbm import LGBMRegressor

print("Pandas:", pd.__version__)
print("NeuralForecast imported OK")
print("LightGBM imported OK")

Using NumPy: 2.0.2
Using Torch: 2.8.0+cu126
Pandas: 2.2.2
NeuralForecast imported OK
LightGBM imported OK


In [ ]:
# =========================
# COLAB PREP & PREPROCESSING (RUN FIRST)
# =========================

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

CONFIG_PREP = {
    "CSV_PATH": "/content/drive/MyDrive/myproject/bike_raw.csv",
    "OUT_DIR": "/content/drive/MyDrive/myproject/results_mdn_nhits",
    "UNIQUE_ID": "series_0",   # use constant label or column name if exists
    "FREQ": "h",
    "TIMESTAMP_COL": None,
    "DATE_COL": None,
    "HOUR_COL": None,
    "DEMAND_COL": None,
    "PARSE_DAYFIRST": False
}

import os
os.makedirs(CONFIG_PREP["OUT_DIR"], exist_ok=True)
CLEAN_PATH = os.path.join(CONFIG_PREP["OUT_DIR"], "bike_clean.csv")

import pandas as pd
import numpy as np

def _load_raw(path: str) -> pd.DataFrame:
    return pd.read_csv(path, low_memory=False)

def _build_hourly_frame(df: pd.DataFrame, cfg) -> pd.DataFrame:
    dem_col = cfg["DEMAND_COL"]
    if dem_col is None:
        for cand in ["y","cnt","demand"]:
            if cand in df.columns:
                dem_col = cand; break
    if dem_col is None:
        raise ValueError("No demand column found.")

    # timestamp detection
    ds = None
    if cfg["TIMESTAMP_COL"] and cfg["TIMESTAMP_COL"] in df.columns:
        ds = pd.to_datetime(df[cfg["TIMESTAMP_COL"]], dayfirst=cfg["PARSE_DAYFIRST"], errors="coerce")
    else:
        dcol = cfg["DATE_COL"] if cfg["DATE_COL"] in df.columns else None
        hcol = cfg["HOUR_COL"] if cfg["HOUR_COL"] in df.columns else None
        if dcol and hcol:
            day = pd.to_datetime(df[dcol], errors="coerce").dt.date
            hour = pd.to_numeric(df[hcol], errors="coerce").fillna(0).astype(int).clip(0,23)
            ds = pd.to_datetime(day.astype(str)) + pd.to_timedelta(hour, unit="h")
        else:
            for cand in ["ds","datetime","timestamp"]:
                if cand in df.columns:
                    ds = pd.to_datetime(df[cand], errors="coerce"); break
    if ds is None: raise ValueError("No usable timestamp found.")

    # unique_id
    uid = str(cfg["UNIQUE_ID"])
    if uid in df.columns: uid_vals = df[uid].astype(str)
    else: uid_vals = [uid]*len(df)

    out = pd.DataFrame({"unique_id": uid_vals, "ds": ds, "y": pd.to_numeric(df[dem_col], errors="coerce")})
    out = out.dropna(subset=["ds"]).groupby(["unique_id","ds"], as_index=False)["y"].sum()

    # fill hourly grid
    frames = []
    for u,g in out.groupby("unique_id"):
        g = g.sort_values("ds")
        idx = pd.date_range(g["ds"].min(), g["ds"].max(), freq=cfg["FREQ"])
        g = g.set_index("ds").reindex(idx)
        g["unique_id"] = u
        g["y"] = g["y"].interpolate(limit_direction="both").fillna(0)
        g = g.reset_index().rename(columns={"index":"ds"})
        frames.append(g)
    out = pd.concat(frames, ignore_index=True)
    return out[["unique_id","ds","y"]].sort_values(["unique_id","ds"]).reset_index(drop=True)

# Run preprocessing
raw = _load_raw(CONFIG_PREP["CSV_PATH"])
clean = _build_hourly_frame(raw, CONFIG_PREP)
clean.to_csv(CLEAN_PATH, index=False)
print(f"[PREP] Wrote cleaned dataset → {CLEAN_PATH}")
display(clean.head(3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-587981335.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ds = pd.to_datetime(df[cand], errors="coerce"); break


[PREP] Wrote cleaned dataset → /content/drive/MyDrive/myproject/results_mdn_nhits/bike_clean.csv


,unique_id,ds,y
0,series_0,2011-01-23 02:00:00,18.0
1,series_0,2011-01-23 03:00:00,5.0
2,series_0,2011-01-23 04:00:00,4.0


In [ ]:
# =========================
# MODEL RUN BLOCK (with NHITS fixes)
# =========================

import os, gc, time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from typing import Tuple
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import PMM, GMM, MAE as NF_MAE

# --- CONFIG (example, assumes CLEAN_PATH is defined by preprocessing) ---
CONFIG = {
    "CLEAN_CSV_PATH": CLEAN_PATH,
    "OUT_DIR": "/content/drive/MyDrive/myproject/results_mdn_nhits/",
    "FREQ": "h",

    "VAL_SIZE": 2*7*24,     # 2 weeks validation
    "TEST_SPLIT_Q": 0.80,   # last 20% timestamps = test
    "HORIZON": 24*14,       # 2 weeks horizon
    "INPUT_SIZE": 24*7,     # 1 week input

    # MDN
    "MDN_MIXES": 10,
    "MDN_HIDDEN": 16,
    "MDN_EPOCHS": 20,
    "MDN_PATIENCE": 20,
    "MDN_LR": 1e-3,

    # NHITS training
    "NHITS_LR": 1e-4,
    "NHITS_STEPS": 1000,
    "NHITS_PATIENCE": 200,
    "NHITS_BATCH": 16,
    "NHITS_WINDOWS_BS": 1,
    "CV_STEP_SIZE": 1,

    "SEEDS_FOR_REPS": [1, 2, 3, 4, 5],
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
}
os.makedirs(CONFIG["OUT_DIR"], exist_ok=True)

# -------------------------
# Helpers
# -------------------------
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def free_mem(*objs):
    for o in objs:
        try: del o
        except: pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def load_clean(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["ds"] = pd.to_datetime(df["ds"])
    return df.sort_values(["unique_id","ds"]).reset_index(drop=True)

def add_calendar(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["Hour"] = df["ds"].dt.hour.astype("int16")
    out["weekday"] = df["ds"].dt.weekday.astype("int8")
    return out

def ensure_dt_key(df: pd.DataFrame, col="ds"):
    if col in df.columns and not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def read_seed_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates=["ds"])
    if "unique_id" in df.columns:
        df["unique_id"] = df["unique_id"].astype(str)
    return df

def align_test_size(test_size_raw: int, h: int, step: int) -> int:
    """Align test_size so that (test_size - h) % step == 0."""
    if test_size_raw < h:
        return h
    rem = (test_size_raw - h) % step
    aligned = test_size_raw - rem
    if aligned < h:
        aligned = h
    return int(aligned)

# -------------------------
# MDN
# -------------------------
class MDN(nn.Module):
    def __init__(self, input_dims, hidden_dims, n_mixes):
        super().__init__()
        self.n_mixes = n_mixes
        self.h1 = nn.Linear(input_dims, hidden_dims)
        self.h2 = nn.Linear(hidden_dims, hidden_dims)
        self.out = nn.Linear(hidden_dims, n_mixes * 3)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.h1(x))
        x = self.act(self.h2(x))
        raw = self.out(x)
        return self._split_params(raw)

    def _split_params(self, raw):
        B, K = raw.size(0), self.n_mixes
        mu = raw[:, :K].view(B, K, 1)
        sigma = torch.exp(raw[:, K:2*K]).view(B, K, 1) + 1e-6
        pis = torch.softmax(raw[:, 2*K:], dim=1)
        return mu, sigma, pis

    @staticmethod
    def nll(mu, sigma, pis, target):
        B, K, _ = mu.shape
        y = target.view(B,1,1).expand(B,K,1)
        dist = torch.distributions.Normal(mu, sigma)
        log_probs = dist.log_prob(y).sum(dim=2)
        weighted = log_probs + torch.log(pis)
        mx = weighted.max(dim=1, keepdim=True).values
        log_sum = mx + torch.log(torch.sum(torch.exp(weighted-mx), dim=1, keepdim=True))
        return -log_sum.mean()

    def mixture_mean_var(self, x):
        mu, sigma, pis = self.forward(x)
        muK = mu.squeeze(-1); sigK = sigma.squeeze(-1)
        mean = torch.sum(pis*muK, dim=1)
        var = torch.sum(pis*(sigK**2 + muK**2), dim=1) - mean**2
        return mean, var.clamp_min(1e-12), muK, sigK, pis

def train_mdn(features, target, n_mixes, hidden, epochs, patience, lr, seed, device):
    set_seed(seed)
    Xtr,Xva,ytr,yva = train_test_split(features,target,test_size=0.2,random_state=seed)
    ysc = StandardScaler()
    ytr_s = ysc.fit_transform(ytr.reshape(-1,1))
    yva_s = ysc.transform(yva.reshape(-1,1))
    Xtr_t, Xva_t = torch.tensor(Xtr,dtype=torch.float32,device=device), torch.tensor(Xva,dtype=torch.float32,device=device)
    ytr_t, yva_t = torch.tensor(ytr_s,dtype=torch.float32,device=device), torch.tensor(yva_s,dtype=torch.float32,device=device)
    model = MDN(Xtr.shape[1], hidden, n_mixes).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)

    best=float("inf"); bad=0; best_state=None
    for ep in range(1,epochs+1):
        model.train(); opt.zero_grad()
        mu,sig,pis=model(Xtr_t); loss=MDN.nll(mu,sig,pis,ytr_t)
        loss.backward(); opt.step()
        model.eval()
        with torch.no_grad():
            mu_v,sig_v,pis_v=model(Xva_t); vloss=MDN.nll(mu_v,sig_v,pis_v,yva_t)
        if vloss.item()<best-1e-4:
            best=vloss.item(); bad=0
            best_state={k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
        else:
            bad+=1
            if bad>=patience: break
    if best_state: model.load_state_dict(best_state)
    free_mem(Xtr_t,Xva_t,ytr_t,yva_t)
    return model, ysc

def mdn_features_for_rows(model, ysc, hours, weekdays, device):
    X = np.column_stack([hours,weekdays]).astype(np.float32)
    Xt = torch.tensor(X,dtype=torch.float32,device=device)
    with torch.no_grad():
        mean_s,var_s,_,_,_ = model.mixture_mean_var(Xt)
    mean_np = mean_s.cpu().numpy().reshape(-1,1)
    var_np  = var_s.cpu().numpy()
    total_mean = ysc.inverse_transform(mean_np).ravel()
    total_std = np.sqrt(var_np) * ysc.scale_[0]
    total_var = total_std**2
    free_mem(Xt)
    return {"total_mean": total_mean, "total_variance": total_var}

# -------------------------
# LGBM
# -------------------------
def add_lags(df, lags=(1,24,168)):
    out = df.sort_values(["unique_id","ds"]).copy()
    for L in lags:
        out[f"lag_{L}"]=out.groupby("unique_id")["y"].shift(L)
    return out

def run_lgbm_once(df_all,use_mdn,validation_cutoff,lags=(1,24,168),seed=42):
    set_seed(seed)
    feat=["Hour","weekday"]+(["total_mean","total_variance"] if use_mdn else [])
    lag_cols=[f"lag_{L}" for L in lags]
    X_cols=feat+lag_cols
    df=add_lags(df_all,lags).copy()
    for c in X_cols+["y"]:
        if c in df.columns: df[c]=pd.to_numeric(df[c],errors="coerce").astype("float32")
    tr=df[df["ds"]<=validation_cutoff].dropna(subset=feat+["y"]).copy()
    te=df[df["ds"]>validation_cutoff].copy()
    model=LGBMRegressor(n_estimators=1000,learning_rate=0.05,
                        subsample=0.9,colsample_bytree=0.9,
                        num_leaves=127,min_child_samples=20,
                        random_state=seed)
    model.fit(tr[X_cols],tr["y"].astype("float32"))
    preds=[]
    for uid,g in te.groupby("unique_id"):
        g=g.sort_values("ds").copy()
        if "lag_1" not in g.columns: g["lag_1"]=np.nan
        g["lag_1"]=pd.to_numeric(g["lag_1"],errors="coerce").astype("float32")
        for i in range(len(g)):
            x_df=g.iloc[[i]][X_cols].astype("float32")
            p=float(model.predict(x_df,validate_features=False)[0])
            preds.append((uid,g.iloc[i]["ds"],p))
            if i+1<len(g):
                g.iat[i+1,g.columns.get_loc("lag_1")]=np.float32(p)
    return pd.DataFrame(preds,columns=["unique_id","ds","yhat"])

# -------------------------
# NHITS
# -------------------------
def build_nhits(loss_obj,seed,use_mdn,cfg):
    return NHITS(
        h=cfg["HORIZON"], input_size=cfg["INPUT_SIZE"],
        loss=loss_obj,
        n_pool_kernel_size=[2,2,2], n_freq_downsample=[24,12,1],
        scaler_type="robust",
        max_steps=cfg["NHITS_STEPS"],
        # early_stop_patience_steps=cfg["NHITS_PATIENCE"],
        learning_rate=cfg["NHITS_LR"],
        batch_size=cfg["NHITS_BATCH"],
        step_size=50,
        windows_batch_size=cfg["NHITS_WINDOWS_BS"],
        num_workers_loader=0, drop_last_loader=False,
        random_seed=seed,
        futr_exog_list=(["total_mean","total_variance"] if use_mdn else None),
        inference_windows_batch_size=1,
        accelerator="auto",
        # precision="16-mixed",
        enable_checkpointing=False, logger=False, enable_model_summary=False,
        detect_anomaly=False
    )

# -------------------------
# MAIN
# -------------------------
def run_all_models_sequential(cfg):
    base=load_clean(cfg["CLEAN_CSV_PATH"])
    base=add_calendar(base)
    threshold_date=base["ds"].quantile(cfg["TEST_SPLIT_Q"])
    VAL_SIZE=int(cfg["VAL_SIZE"])
    TEST_SIZE=base[base["ds"]>threshold_date]["ds"].nunique()
    unique_dates=sorted(base["ds"].unique())
    training_cutoff=unique_dates[-(VAL_SIZE+TEST_SIZE)]
    validation_cutoff=unique_dates[-TEST_SIZE]

    base["is_train"]=base["ds"]<=training_cutoff
    base["is_val"]=(base["ds"]>training_cutoff)&(base["ds"]<=validation_cutoff)
    base["is_test"]=base["ds"]>validation_cutoff

    dense_test=ensure_dt_key(base[base["is_test"]][["unique_id","ds","y"]].copy())
    dense_test_path=os.path.join(cfg["OUT_DIR"],"test_actuals.csv")
    dense_test.to_csv(dense_test_path,index=False)

    # --- MDN ---
    Xtr=base.loc[base["is_train"],["Hour","weekday"]].to_numpy(np.float32)
    ytr=base.loc[base["is_train"],"y"].to_numpy(np.float32)
    mdn,ysc=train_mdn(Xtr,ytr,cfg["MDN_MIXES"],cfg["MDN_HIDDEN"],cfg["MDN_EPOCHS"],
                      cfg["MDN_PATIENCE"],cfg["MDN_LR"],cfg["SEEDS_FOR_REPS"][0],cfg["DEVICE"])
    feats=mdn_features_for_rows(mdn,ysc,base["Hour"].to_numpy(),base["weekday"].to_numpy(),cfg["DEVICE"])
    base["total_mean"]=feats["total_mean"]; base["total_variance"]=feats["total_variance"]
    free_mem(mdn,ysc)

    df_with=base[["unique_id","ds","y","total_mean","total_variance","Hour","weekday"]].copy().sort_values(["unique_id","ds"])
    df_no=base[["unique_id","ds","y","Hour","weekday"]].copy().sort_values(["unique_id","ds"])

    # ---------- LGBM ----------
    for label,use_mdn in [("lgbm_nomdn",False),("lgbm_mdn",True)]:
        print(f"\n[RUN] {label}")
        for seed in cfg["SEEDS_FOR_REPS"]:
            preds=run_lgbm_once(df_with if use_mdn else df_no,use_mdn,validation_cutoff,seed=seed)
            preds.to_csv(os.path.join(cfg["OUT_DIR"],f"test_forecasts_{label}_seed{seed}.csv"),index=False)
            free_mem(preds)

    # ---------- NHITS ----------
    nhits_variants=[
        ("nhits_mae_nomdn", NF_MAE(), False),
        ("nhits_mae_mdn", NF_MAE(), True),
        ("nhits_gmm_nomdn", GMM(n_components=15, quantiles=[.5]), False),
        ("nhits_pmm_nomdn", PMM(n_components=15, quantiles=[.5]), False),
    ]
    H=int(cfg["HORIZON"])
    STEP_DEFAULT=int(cfg["CV_STEP_SIZE"])
    TEST_SIZE_RAW=TEST_SIZE
    step_size_cv=STEP_DEFAULT
    if TEST_SIZE_RAW-H<STEP_DEFAULT and TEST_SIZE_RAW>=H: step_size_cv=1
    TEST_SIZE_CV=align_test_size(TEST_SIZE_RAW,H,step_size_cv)
    if TEST_SIZE_CV<H: TEST_SIZE_CV=H

    for label,loss_obj,use_mdn in nhits_variants:
        print(f"\n[RUN] {label}")
        cols_in=["unique_id","ds","y"]+(["total_mean","total_variance"] if use_mdn else [])
        df_in=(df_with if use_mdn else df_no)[cols_in]

        for seed in cfg["SEEDS_FOR_REPS"]:
            set_seed(seed)
            model=build_nhits(loss_obj,seed,use_mdn,cfg)
            nf=NeuralForecast(models=[model],freq=cfg["FREQ"])

            Y_hat=nf.cross_validation(
                df=df_in,val_size=int(cfg["VAL_SIZE"]),
                test_size=TEST_SIZE_CV,n_windows=None,
                step_size=step_size_cv,verbose=False
            ).reset_index()

            ycol="NHITS" if "NHITS" in Y_hat.columns else [c for c in Y_hat.columns if c.upper()=="NHITS"][0]
            Y_seed=Y_hat.loc[Y_hat["ds"]>validation_cutoff,["unique_id","ds",ycol]].copy()
            Y_seed=Y_seed.rename(columns={ycol:"yhat"})
            Y_seed=ensure_dt_key(Y_seed,"ds")
            Y_seed.to_csv(os.path.join(cfg["OUT_DIR"],f"test_forecasts_{label}_seed{seed}.csv"),index=False)
            free_mem(Y_hat,Y_seed,nf,model)

    print("\n[OK] All models finished. Forecasts saved.")
    return dense_test_path

# --- Run all models
dense_test_path=run_all_models_sequential(CONFIG)
print("Dense test (actuals) saved at:", dense_test_path)


[RUN] lgbm_nomdn
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 796
[LightGBM] [Info] Number of data points in the train set: 13612, number of used features: 5
[LightGBM] [Info] Start training from score 178.757236
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 796
[LightGBM] [Info] Number of data points in the train set: 13612, number of used features: 5
[LightGBM] [Info] Start training from score 178.757236
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 796
[LightGBM] [Info] Number of data points in the train set: 13612, number of used features: 5
[LightGBM] 

INFO:lightning_fabric.utilities.seed:Seed set to 1



[RUN] nhits_mae_nomdn


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/neuralforecast/core.py:209: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(



[OK] All models finished. Forecasts saved.
Dense test (actuals) saved at: /content/drive/MyDrive/myproject/results_mdn_nhits/test_actuals.csv


In [ ]:
# =========================
# MAPE CALCULATION (Daily-first)
# =========================
import os
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import t

# --- config (match your run block) ---
OUT_DIR = "/content/drive/MyDrive/myproject/results_mdn_nhits"
SEEDS   = [1,2,3,4,5]

MODEL_LABELS = [
    "lgbm_nomdn",
    "lgbm_mdn",
    "nhits_mae_nomdn",
    "nhits_mae_mdn",
    "nhits_gmm_nomdn",
    "nhits_pmm_nomdn",
]

# --- load the fixed test grid (actuals only) ---
dense_test = pd.read_csv(os.path.join(OUT_DIR, "test_actuals.csv"), parse_dates=["ds"])
dense_test["unique_id"] = dense_test["unique_id"].astype(str)
dense_test = (dense_test
              .drop_duplicates(subset=["unique_id","ds"], keep="last")
              .sort_values(["unique_id","ds"])
              .reset_index(drop=True))

# Helpers
def daily_agg(df, y_col="y", yhat_col="yhat"):
    """Aggregate to daily totals per unique_id; return y_daily, yhat_daily, and daily MAPE series."""
    g = df.copy()
    g["date"] = g["ds"].dt.date
    daily = (g.groupby(["unique_id","date"], as_index=False)[[y_col, yhat_col]].sum()
               .rename(columns={y_col:"y_daily", yhat_col:"yhat_daily"}))
    # daily MAPE, ignore zero-actual days
    m = daily["y_daily"].to_numpy(dtype=float)
    p = daily["yhat_daily"].to_numpy(dtype=float)
    mask = np.isfinite(m) & np.isfinite(p) & (m != 0.0)
    daily["MAPE"] = np.where(mask, 100.0 * np.abs((m - p) / m), np.nan)
    return daily.dropna(subset=["MAPE"])

def ci95_from_samples(vals):
    """Return (mean, lower, upper) 95% CI using Student t (valid for n>=2)."""
    vals = np.asarray(vals, dtype=float)
    vals = vals[np.isfinite(vals)]
    n = len(vals)
    if n == 0:
        return (np.nan, np.nan, np.nan)
    mean = float(np.mean(vals))
    if n == 1:
        return (mean, np.nan, np.nan)
    sd = float(np.std(vals, ddof=1))
    se = sd / sqrt(n)
    tcrit = float(t.ppf(0.975, df=n-1))
    return (mean, mean - tcrit*se, mean + tcrit*se)

# --- per-model results we’ll collect ---
per_seed_summary_rows = []   # one row per (model, seed)
model_summary_rows    = []   # one row per model (aggregated across seeds)

for label in MODEL_LABELS:
    seed_means = []  # mean of daily MAPE per seed (for across-seed CI)
    for s in SEEDS:
        # load forecast for this seed
        fp = os.path.join(OUT_DIR, f"test_forecasts_{label}_seed{s}.csv")
        if not os.path.exists(fp):
            print(f"[WARN] Missing forecast file: {fp} — skipping this seed.")
            continue
        pred = pd.read_csv(fp, parse_dates=["ds"])
        pred["unique_id"] = pred["unique_id"].astype(str)
        pred = (pred
                .drop_duplicates(subset=["unique_id","ds"], keep="last")
                .sort_values(["unique_id","ds"])
                .reset_index(drop=True))

        # left-merge onto test grid (keeps length & timestamps aligned to test only)
        merged = dense_test.merge(
            pred.rename(columns={"yhat":"yhat"}),  # name is already yhat, but keep explicit
            on=["unique_id","ds"], how="left"
        )

        # aggregate to daily and compute daily MAPE
        daily = daily_agg(merged, y_col="y", yhat_col="yhat")
        if daily.empty:
            print(f"[WARN] No overlapping daily rows for {label} seed {s}; skipping.")
            continue

        # per-seed mean and CI across days
        mean_d, lo_d, hi_d = ci95_from_samples(daily["MAPE"].to_numpy())
        per_seed_summary_rows.append({
            "Model": label,
            "Seed": s,
            "Days_Evaluated": int(len(daily)),
            "Mean_Daily_MAPE_%": mean_d,
            "CI95_lower_%": lo_d,
            "CI95_upper_%": hi_d,
        })
        seed_means.append(mean_d)

    # aggregate across seeds (mean of per-seed means)
    if len(seed_means) > 0:
        m, lo, hi = ci95_from_samples(seed_means)
        model_summary_rows.append({
            "Model": label,
            "Seeds_Used": len(seed_means),
            "Mean_of_SeedMeans_MAPE_%": m,
            "CI95_lower_%": lo,
            "CI95_upper_%": hi,
            "SeedMeans_List": np.round(seed_means, 4).tolist(),
        })
    else:
        model_summary_rows.append({
            "Model": label,
            "Seeds_Used": 0,
            "Mean_of_SeedMeans_MAPE_%": np.nan,
            "CI95_lower_%": np.nan,
            "CI95_upper_%": np.nan,
            "SeedMeans_List": [],
        })

# --- save outputs ---
per_seed_df  = pd.DataFrame(per_seed_summary_rows).sort_values(["Model","Seed"]).reset_index(drop=True)
model_df     = pd.DataFrame(model_summary_rows).sort_values("Mean_of_SeedMeans_MAPE_%").reset_index(drop=True)

per_seed_path = os.path.join(OUT_DIR, "mape_daily_per_seed.csv")
model_path    = os.path.join(OUT_DIR, "mape_daily_summary_by_model.csv")
per_seed_df.to_csv(per_seed_path, index=False)
model_df.to_csv(model_path, index=False)

# --- pretty print ---
print("\n=== Per-seed (mean of daily MAPE) with 95% CI over days ===")
if not per_seed_df.empty:
    for _, r in per_seed_df.iterrows():
        print(f"{r['Model']:18s} | seed {int(r['Seed'])}: "
              f"{r['Mean_Daily_MAPE_%']:.3f}% "
              f"(CI95 {r['CI95_lower_%']:.3f}–{r['CI95_upper_%']:.3f}, "
              f"days={int(r['Days_Evaluated'])})")
else:
    print("No seeds evaluated.")

print("\n=== Across-seed summary (mean of per-seed means) with 95% CI across seeds ===")
for _, r in model_df.iterrows():
    print(f"{r['Model']:18s}: {r['Mean_of_SeedMeans_MAPE_%']:.3f}%  "
          f"(CI95 {r['CI95_lower_%']:.3f}–{r['CI95_upper_%']:.3f}, "
          f"seeds={int(r['Seeds_Used'])})  SeedMeans={r['SeedMeans_List']}")
print("\nSaved:")
print(" -", per_seed_path)
print(" -", model_path)


=== Per-seed (mean of daily MAPE) with 95% CI over days ===
lgbm_mdn           | seed 1: 32.295% (CI95 17.365–47.225, days=142)
lgbm_mdn           | seed 2: 33.764% (CI95 18.294–49.234, days=142)
lgbm_mdn           | seed 3: 32.378% (CI95 17.881–46.875, days=142)
lgbm_mdn           | seed 4: 32.369% (CI95 18.247–46.491, days=142)
lgbm_mdn           | seed 5: 32.926% (CI95 18.187–47.664, days=142)
lgbm_nomdn         | seed 1: 33.814% (CI95 17.883–49.746, days=142)
lgbm_nomdn         | seed 2: 33.814% (CI95 17.883–49.746, days=142)
lgbm_nomdn         | seed 3: 33.814% (CI95 17.883–49.746, days=142)
lgbm_nomdn         | seed 4: 33.814% (CI95 17.883–49.746, days=142)
lgbm_nomdn         | seed 5: 33.814% (CI95 17.883–49.746, days=142)
nhits_gmm_nomdn    | seed 1: 24.403% (CI95 8.136–40.670, days=142)
nhits_gmm_nomdn    | seed 2: 27.498% (CI95 10.442–44.555, days=142)
nhits_gmm_nomdn    | seed 3: 23.815% (CI95 8.535–39.095, days=142)
nhits_gmm_nomdn    | seed 4: 26.472% (CI95 10.365–42.578,